In [1]:
import numpy as np
import pandas as pd

In [3]:
test = pd.read_csv('Test (5).csv')
test_content_list = test['content'].tolist()

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("flax-community/bert-swahili-news-classification", model_max_length=1000)
model = AutoModelForSequenceClassification.from_pretrained("flax-community/bert-swahili-news-classification")

tokenizer_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/17.9k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

/home/unicconaiadmin/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
import torch

device = "cpu"
device

'cpu'

In [9]:
torch.cuda.is_available()

True

In [17]:
tokens = tokenizer(test_content_list, padding=True, truncation=True, return_tensors='pt').to(device)

model.eval()
with torch.no_grad():
    outputs = model(**tokens)

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
probabilities_cpu = probabilities.cpu().numpy()

In [47]:
columns = ['Biashara', 'Kitaifa', 'michezo', 'Kimataifa', 'Burudani', 'afya-health']

df = pd.DataFrame(probabilities_cpu, columns=columns)

In [48]:
df['swahili_id'] = test['swahili_id']
new_order = ['swahili_id', 'Kitaifa', 'michezo', 'Biashara', 'Kimataifa', 'Burudani', 'afya-health']
df = df[new_order]

df = df.drop(['afya-health'], axis=1)
df.to_csv('old_swahili_submission_1.csv', index=False)
df

,swahili_id,Kitaifa,michezo,Biashara,Kimataifa,Burudani
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,0.998397,0.000485,0.000468,0.000174,0.000029
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,0.032023,0.964994,0.000684,0.000726,0.001185
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,0.478689,0.000448,0.517480,0.002858,0.000251
3,00868eeee349e286303706ef0ffd851f39708d37,0.958453,0.001199,0.000172,0.002239,0.000409
4,00a5cb12d3058dcf2e42f277eee599992db32412,0.969958,0.000601,0.029053,0.000149,0.000054
...,...,...,...,...,...,...
1283,feb4b0ae88524c9cee3e50f2301d84a235f3c607,0.996162,0.000280,0.000321,0.000495,0.000036
1284,fecf3f14f47237e02721ed4baa4eb6c11abd239f,0.000329,0.999022,0.000269,0.000108,0.000228
1285,feed09e13586d12139cd59bc20996a4d29706606,0.203288,0.791683,0.000487,0.001424,0.002423
1286,ffc0bca6fde8fa0cce1ac3a7b0d746603d441bf8,0.012799,0.985916,0.000108,0.000328,0.000761


In [29]:
new_df = df[["Kitaifa","michezo","Biashara","Kimataifa","Burudani" ]] + pd.read_csv("/home/unicconaiadmin/Music1/Msa/ensamble_swahili.csv")[["Kitaifa","michezo","Biashara","Kimataifa","Burudani"]]  + pd.read_csv("/home/unicconaiadmin/Music1/Msa/swahili_submission.csv")[["Kitaifa","michezo","Biashara","Kimataifa","Burudani"]] /3

In [30]:
df[["Kitaifa","michezo","Biashara","Kimataifa","Burudani" ]] = new_df[["Kitaifa","michezo","Biashara","Kimataifa","Burudani" ]] 

In [31]:
df.to_csv("ensamble_swahili.csv",index=False)

In [32]:
df

,swahili_id,Kitaifa,michezo,Biashara,Kimataifa,Burudani
0,ae3baa6c34aa523fd2aa4de3c89448efff922311,2.679639,0.002165,0.150148,0.000792,0.000221
1,c4ee26a3ade8064a2ec494996e836900fd32dd8e,2.828931,0.001238,0.001768,0.000267,0.000082
2,58aee3aa1d94554ff57e6a053dbd60658e4890ff,0.176864,2.650224,0.000761,0.001135,0.003733
3,00579c2307b5c11003d21c40c3ecff5e922c3fd8,1.261515,0.001256,1.558764,0.010096,0.000851
4,c83e9738ae5d1790ee85b99863deb734e7614c52,2.787808,0.002674,0.041705,0.000582,0.000146
...,...,...,...,...,...,...
1025,53332d2e0363433b554581a07239a648c5bccaff,2.797139,0.001482,0.033422,0.000931,0.000147
1026,9362cab7b1f02ae1e56b9b62b23f16bddc55b1b5,0.000430,0.001309,2.831223,0.000193,0.000079
1027,6cd87e2b10f1c509ad970c519843efd4fea963fa,0.000502,0.004172,2.828247,0.000171,0.000105
1028,361806eaeeb8e109ee70a5a7682afb4c62db6a6b,0.085819,2.741611,0.000324,0.001319,0.003929


In [18]:
new_df

,Kitaifa,michezo,Biashara,Kimataifa,Burudani
0,1.415822,0.001179,0.082310,0.000383,0.000111
1,1.497593,0.000790,0.001036,0.000123,0.000040
2,0.106191,1.390579,0.000431,0.000639,0.001825
3,0.738801,0.000674,0.755468,0.004273,0.000377
4,1.478825,0.001419,0.019177,0.000287,0.000071
...,...,...,...,...,...
1025,1.481075,0.000683,0.017612,0.000442,0.000075
1026,0.000242,0.000704,1.498864,0.000098,0.000041
1027,0.000282,0.002294,1.497207,0.000087,0.000055
1028,0.052088,1.444700,0.000177,0.000735,0.002112
